### Year 5, Topology 6 - Limit Checking - Overloading, Voltage limits

In [106]:
import pandas as pd
import altair as alt

##### Loading

##### Percentage Loading in Branches 

Calculation related can be found in PSSE PAGVI Manual Chapter 6 Power system network simulations - Section 6.6.7 Activity Rate 

%I = MVA(actual)/(MVA(rated)*Vpu)

Vpu -> Bus Voltage at from end 

In [107]:
list_all = []
list_gens = [0,50,100]
list_lsc = ['LLS','RLS','HLS']
list_gen_hydro = [400,500,600] #dry, average, wet hydological scenarios
list_gen_wind = [0,40,75] # for 3 wind gust scenarios
for gen in list_gens:
    for gen_hy in list_gen_hydro:
        for lsc in list_lsc:
            for gen_wi in list_gen_wind:
                file_csv_out = 'loading/'+'savnw_sol_' + str(gen) +'_hy_' +str(gen_hy) +'_wi_' + str(gen_wi) +'_' + str(lsc)+'_l.csv'
                data_load = pd.read_csv(file_csv_out,header=1, skiprows=0)
                # Extracted data contains branch loading from the metered and non-metered ends. The data is filtered to obtain the branch loading from the metered end.
                data_fil = data_load
                data_fil['From Bus V'] = data_fil['From Bus Ext'].str.extract(r'(\d+\.\d+)') 
                data_fil['To Bus V'] = data_fil['To Bus Ext'].str.extract(r'(\d+\.\d+)')
                data_fil['Scenario']= 'Solar = ' + str(gen) + ' MW, ' + 'Hydro = ' + str(gen_hy) + ' MW, ' + 'Wind = ' + str(gen_wi) + ' MW, '  + lsc.upper()
                list_all.append(data_fil)
data_all = pd.concat(list_all).reset_index(drop=True)
data_all = data_all[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID',
       'Non Metered Bus', ' Branch I', 'Branch Flow MW', 'Branch Flow MVAR',
       'Branch MVA', 'MW Loss', 'MVAR Loss', 'Rate', '%I', '%MVA',
       'From Bus V', 'To Bus V']]

##### Line and Transformer Loading

Transformer and branch loading are categorised to following brackets based on the range to which it falls into 
* Loading less than 25%
* Loading 25% to 50%
* Loading 50% to 70%
* Loading 70% to 80%
* Loading 80% to 85%
* Loading 85% to 90%
* Loading 90% to 95%
* Loading 95% to 100%
* Loading 100% and above

Separating transformer and line branch loadings

In [108]:
transformer_loading = data_all[data_all['From Bus V']!=data_all['To Bus V']].reset_index(drop=True)
branch_loading = data_all[data_all['From Bus V'] == data_all['To Bus V']].reset_index(drop=True)

Defining loading brackets for transformer and line branches 

In [109]:
data = transformer_loading[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID','Branch Flow MW', 'Branch Flow MVAR','Branch MVA', 'Rate','%MVA']]
data['Loading'] = ''
data.loc[(data['%MVA'] >100), 'Loading'] = '%MVA>100'
data.loc[(data['%MVA'] <100), 'Loading'] = '95<%MVA<100'
data.loc[(data['%MVA'] <95), 'Loading'] = '90<%MVA<95'
data.loc[(data['%MVA'] <90), 'Loading'] = '85<%MVA<90'
data.loc[(data['%MVA'] <85), 'Loading'] = '80<%MVA<85'
data.loc[(data['%MVA'] <80), 'Loading'] = '70<%MVA<80'
data.loc[(data['%MVA'] <70), 'Loading'] = '50<%MVA<70'
data.loc[(data['%MVA'] <50), 'Loading'] = '25<%MVA<50'
data.loc[(data['%MVA'] <25), 'Loading'] = '%MVA<25'

<ipython-input-109-7f16dd71236d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Loading'] = ''


In [110]:
data_l = branch_loading[['Scenario','From Bus', 'From Bus Ext', 'To Bus', 'To Bus Ext', 'Ckt ID','Branch Flow MW', 'Branch MVA', 'Rate','%I']]
data_l['Loading'] = ''
data_l.loc[(data_l['%I'] >100), 'Loading'] = '%I>100'
data_l.loc[(data_l['%I'] <100), 'Loading'] = '95<%I<100'
data_l.loc[(data_l['%I'] <95), 'Loading'] = '90<%I<95'
data_l.loc[(data_l['%I'] <90), 'Loading'] = '85<%I<90'
data_l.loc[(data_l['%I'] <85), 'Loading'] = '80<%I<85'
data_l.loc[(data_l['%I'] <80), 'Loading'] = '70<%I<80'
data_l.loc[(data_l['%I'] <70), 'Loading'] = '50<%I<70'
data_l.loc[(data_l['%I'] <50), 'Loading'] = '25<%I<50'
data_l.loc[(data_l['%I'] <25), 'Loading'] = '%I<25'

<ipython-input-110-abcf08d21c83>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_l['Loading'] = ''


##### Histogram Transformer Branch Loading 

A histogram for each scenario is plotted to check the no of transformer branches in the above defined loading bracket 

In [111]:
alt.Chart(data).mark_bar().encode(
    alt.Y('Loading',scale = alt.Scale(domain=['%MVA>100','95<%MVA<100','90<%MVA<95','85<%MVA<90','80<%MVA<85','70<%MVA<80','50<%MVA<70','25<%MVA<50','%MVA<25'])),
    alt.X('count()')).properties(height = 120, width = 250).facet('Scenario',columns=5).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that for most of scenarios the transformer branch loading falls into a bracket of loading between 50 to 70 percentage. 

##### Histogram Line Branch Loading 

A histogram is plotted to check the no of line branches in the above defined loading bracket 

In [112]:
alt.Chart(data_l).mark_bar().encode(
    alt.Y('Loading', scale =alt.Scale(domain=['%I>100','95<%I<100','90<%I<95','85<%I<90','80<%I<85','70<%I<80','50<%I<70','25<%I<50','%I<25'])),
    alt.X('count()')).properties(height = 120, width = 250).facet('Scenario',columns=5).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that for most of scenarios the line branch loading falls into a bracket of loading between 25% to 50%

In the following section the lines and transformer branches with loading greater than 80% is filtered 

##### Checking transformer loading greater than 80% 

In [113]:
transformer_loading = data_all[data_all['From Bus V']!=data_all['To Bus V']].reset_index(drop=True)
transformer_overload  = transformer_loading[transformer_loading['%MVA'] > 80.0].reset_index(drop=True)
transformer_overload['Branch'] = transformer_overload['From Bus'].astype(str) + ' ' + transformer_overload['From Bus Ext'] + ' to ' + \
transformer_overload['To Bus'].astype(str) + ' '+ transformer_overload['To Bus Ext'] + ' ' + transformer_overload['Ckt ID'].astype(str)
list_branch = transformer_overload['Branch'].unique()
print(list_branch)

['3018 CATDOG_G    13.800 to 3008 CATDOG      230.00 1'
 '206 URBGEN      18.000 to 205 SUB230      230.00 1'
 '205 SUB230      230.00 to 206 URBGEN      18.000 1'
 '211 HYDRO_G     20.000 to 201 HYDRO       500.00 1'
 '212 HYDRO_N     20.000 to 201 HYDRO       500.00 1']


In [114]:
transformer_overload

,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V,Branch
0,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, LLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4805,90.00,80.00,120.41,0.03,11.21,150.0,76.58,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
1,"Solar = 0 MW, Hydro = 400 MW, Wind = 40 MW, LLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4805,90.00,80.00,120.42,0.03,11.21,150.0,76.57,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
2,"Solar = 0 MW, Hydro = 400 MW, Wind = 75 MW, LLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4805,90.00,80.00,120.42,0.03,11.21,150.0,76.57,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
3,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, RLS",206,URBGEN 18.000,205,SUB230 230.00,1,206,29981,754.37,600.00,963.88,2.27,116.47,1150.0,81.28,83.82,18.000,230.00,206 URBGEN 18.000 to 205 SUB230 230....
4,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, RLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,4899,90.00,80.00,120.41,0.03,11.66,150.0,78.07,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,"Solar = 100 MW, Hydro = 600 MW, Wind = 40 MW, HLS",212,HYDRO_N 20.000,201,HYDRO 500.00,1,212,17147,600.01,231.78,643.22,2.47,74.98,800.0,74.25,80.40,20.000,500.00,212 HYDRO_N 20.000 to 201 HYDRO 500....
122,"Solar = 100 MW, Hydro = 600 MW, Wind = 40 MW, HLS",3018,CATDOG_G 13.800,3008,CATDOG 230.00,1,3018,5054,90.00,80.00,120.41,0.03,12.41,150.0,80.54,80.28,13.800,230.00,3018 CATDOG_G 13.800 to 3008 CATDOG 23...
123,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, HLS",211,HYDRO_G 20.000,201,HYDRO 500.00,1,211,17161,600.01,233.85,643.97,2.47,75.10,800.0,74.31,80.50,20.000,500.00,211 HYDRO_G 20.000 to 201 HYDRO 500....
124,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, HLS",212,HYDRO_N 20.000,201,HYDRO 500.00,1,212,17161,600.01,233.85,643.97,2.47,75.10,800.0,74.31,80.50,20.000,500.00,212 HYDRO_N 20.000 to 201 HYDRO 500....


##### Transformer branch overload plots

In [115]:
tran_fil = transformer_overload[transformer_overload['Branch']!='3018 CATDOG_G    13.800 to 3008 CATDOG      230.00 1']
tran_fil_no = transformer_overload[transformer_overload['Branch']=='3018 CATDOG_G    13.800 to 3008 CATDOG      230.00 1']

In [116]:
alt.Chart(tran_fil).mark_line().encode(
    alt.X('%MVA', scale=alt.Scale(zero=False)),
    alt.Y('Scenario',axis = alt.Axis(labelLimit= 250, titleX=-260), sort='-x'),
    alt.Color('Branch', legend=None)).properties(height = 210, width =350).facet('Branch',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

In [117]:
alt.Chart(tran_fil_no).mark_line().encode(
    alt.X('%MVA', scale=alt.Scale(zero=False)),
    alt.Y('Scenario',axis = alt.Axis(labels=False, ticks = False), title='All Study Scenarios'),
    alt.Color('Branch', legend=None)).properties(height = 120, width =400).facet('Branch',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

##### Checking branch loading greater than 80% 

In [118]:
branch_loading = data_all[data_all['From Bus V'] == data_all['To Bus V']].reset_index(drop=True)
branch_overload = branch_loading[branch_loading['%I'] > 80.0].reset_index(drop=True)
branch_overload['Branch'] = branch_overload['From Bus'].astype(str) + ' ' + branch_overload['From Bus Ext'] + ' to ' + \
branch_overload['To Bus'].astype(str) + ' '+ branch_overload['To Bus Ext'] + ' ' + branch_overload['Ckt ID'].astype(str)
list_branch = branch_overload['Branch'].unique()
print(list_branch)

['154 DOWNTN      230.00 to 203 EAST230     230.00 1'
 '203 EAST230     230.00 to 154 DOWNTN      230.00 1'
 '154 DOWNTN      230.00 to 205 SUB230      230.00 1'
 '205 SUB230      230.00 to 154 DOWNTN      230.00 1'
 '154 DOWNTN      230.00 to 153 MID230      230.00 1'
 '153 MID230      230.00 to 154 DOWNTN      230.00 1'
 '203 EAST230     230.00 to 205 SUB230      230.00 1'
 '203 EAST230     230.00 to 205 SUB230      230.00 2'
 '205 SUB230      230.00 to 203 EAST230     230.00 1'
 '205 SUB230      230.00 to 203 EAST230     230.00 2'
 '204 SUB500      500.00 to 201 HYDRO       500.00 1']


In [119]:
branch_overload

,Scenario,From Bus,From Bus Ext,To Bus,To Bus Ext,Ckt ID,Non Metered Bus,Branch I,Branch Flow MW,Branch Flow MVAR,Branch MVA,MW Loss,MVAR Loss,Rate,%I,%MVA,From Bus V,To Bus V,Branch
0,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, LLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,410,-150.97,-48.99,158.72,1.05,10.50,200.0,81.73,79.36,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....
1,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, LLS",203,EAST230 230.00,154,DOWNTN 230.00,1,203,402,152.02,49.80,159.97,1.05,10.50,200.0,80.20,79.98,230.00,230.00,203 EAST230 230.00 to 154 DOWNTN 230....
2,"Solar = 0 MW, Hydro = 400 MW, Wind = 40 MW, LLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,411,-151.40,-48.97,159.12,1.06,10.55,200.0,81.93,79.56,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....
3,"Solar = 0 MW, Hydro = 400 MW, Wind = 40 MW, LLS",203,EAST230 230.00,154,DOWNTN 230.00,1,203,403,152.46,49.83,160.39,1.06,10.55,200.0,80.41,80.20,230.00,230.00,203 EAST230 230.00 to 154 DOWNTN 230....
4,"Solar = 0 MW, Hydro = 400 MW, Wind = 75 MW, LLS",154,DOWNTN 230.00,203,EAST230 230.00,1,203,412,-151.82,-48.86,159.49,1.06,10.61,200.0,82.13,79.74,230.00,230.00,154 DOWNTN 230.00 to 203 EAST230 230....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, HLS",203,EAST230 230.00,154,DOWNTN 230.00,1,203,516,179.56,74.22,194.29,1.72,17.22,200.0,102.81,97.15,230.00,230.00,203 EAST230 230.00 to 154 DOWNTN 230....
234,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, HLS",203,EAST230 230.00,205,SUB230 230.00,1,203,402,144.95,43.55,151.36,1.30,11.71,200.0,80.09,75.68,230.00,230.00,203 EAST230 230.00 to 205 SUB230 230....
235,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, HLS",203,EAST230 230.00,205,SUB230 230.00,2,203,402,144.95,43.55,151.36,1.30,11.71,200.0,80.09,75.68,230.00,230.00,203 EAST230 230.00 to 205 SUB230 230....
236,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, HLS",205,SUB230 230.00,203,EAST230 230.00,1,203,407,-143.65,-38.78,148.80,1.30,11.71,200.0,81.11,74.40,230.00,230.00,205 SUB230 230.00 to 203 EAST230 230....


##### Transmission line overload plots

In [120]:
branch_fil = branch_overload[(branch_overload['Branch'] !='154 DOWNTN      230.00 to 203 EAST230     230.00 1') & (branch_overload['Branch'] !='203 EAST230     230.00 to 154 DOWNTN      230.00 1') ].reset_index(drop = True)

In [121]:
branch_fil_1 = branch_overload[(branch_overload['Branch'] =='154 DOWNTN      230.00 to 203 EAST230     230.00 1') | (branch_overload['Branch'] =='203 EAST230     230.00 to 154 DOWNTN      230.00 1') ].reset_index(drop = True)

In [122]:
alt.Chart(branch_fil).mark_line().encode(
    alt.X('%I', scale=alt.Scale(zero=False)),
    alt.Y('Scenario',axis = alt.Axis(labelLimit= 250, titleX=-260), sort='-x'),
    alt.Color('Branch', legend=None)).properties(height = 200, width =200).facet('Branch',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

In [123]:
alt.Chart(branch_fil_1).mark_line().encode(
    alt.X('%I', scale=alt.Scale(zero=False)),
    alt.Y('Scenario',axis = alt.Axis(labelLimit= 250, titleX=-260),sort='-x'),
    alt.Color('Branch', legend=None)).properties(height = 750, width =250).facet('Branch',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

##### Bus Voltage

In [124]:
list_volt = []
list_gens = [0,50,100]
list_lsc = ['LLS','RLS','HLS']
list_gen_hydro = [400,500,600] #dry, average, wet hydological scenarios
list_gen_wind = [0,40,75] # for 3 wind gust scenarios
for gen in list_gens:
    for gen_hy in list_gen_hydro:
        for lsc in list_lsc:
            for gen_wi in list_gen_wind:
                file_csv_in = 'volt/'+'savnw_sol_' + str(gen) +'_hy_' +str(gen_hy) +'_wi_' + str(gen_wi) +'_' + str(lsc)+'_volt.csv'
                data_volt = pd.read_csv(file_csv_in,header=1, skiprows=0)
                data_volt['Scenario']= 'Solar = ' + str(gen) + ' MW, ' + 'Hydro = ' + str(gen_hy) + ' MW, ' + 'Wind = ' + str(gen_wi) + ' MW, '  + lsc.upper()
                list_volt.append(data_volt)
data_all_volt = pd.concat(list_volt).reset_index(drop=True)
data_all_volt = data_all_volt[['Scenario','Bus Number', 'Bus Name', 'Area', 'Base Voltage', 'Bus Voltage(PU)','Bus Voltage(kV)']]

Similar to branch loading categorization into brackets of loadings, voltages are also categorised into brackets

In [125]:
data_v = data_all_volt
data_v['Voltage Range'] = ''
data_v.loc[(data_v['Bus Voltage(PU)'] >1.1), 'Voltage Range'] = 'Vbus>1.1'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.1), 'Voltage Range'] = '1.05<Vbus<1.1'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.05), 'Voltage Range'] = '1.0<Vbus<1.05'
data_v.loc[(data_v['Bus Voltage(PU)'] <1.0), 'Voltage Range'] = '0.95<Vbus<1.0'
data_v.loc[(data_v['Bus Voltage(PU)'] <0.95), 'Voltage Range'] = '0.95<Vbus<0.9'
data_v.loc[(data_v['Bus Voltage(PU)'] <0.9), 'Voltage Range'] = 'Vbus<0.9'

##### Histogram Bus Voltages

A histogram is plotted to check the no of Buses in each of the above defined brackets 

In [126]:
alt.Chart(data_v).mark_bar().encode(
    alt.Y('Voltage Range', scale =alt.Scale(domain=['Vbus>1.1','1.05<Vbus<1.1','1.0<Vbus<1.05','0.95<Vbus<1.0','0.95<Vbus<0.9','Vbus<0.9'])),
    alt.X('count()')).properties(height = 100,width = 200).facet('Scenario',columns=6).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

It can be seen that most of the buses have voltages in the bracket of 1.0 PU and 1.05 PU

In the following section, the buses out of upper and lower normal operating range is filtered and reported 

##### Voltage Upper Limits 

In [127]:
volt_high = data_all_volt[data_all_volt['Bus Voltage(PU)']>1.05].reset_index(drop=True)
list_bus = list(volt_high['Bus Number'].unique())
list_bus

[206, 211, 212]

In [128]:
volt_high

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV),Voltage Range
0,"Solar = 0 MW, Hydro = 400 MW, Wind = 75 MW, LLS",206,URBGEN,2,18.0,1.051,18.910,1.05<Vbus<1.1
1,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, HLS",211,HYDRO_G,2,20.0,1.066,21.326,1.05<Vbus<1.1
2,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, HLS",212,HYDRO_N,2,20.0,1.066,21.326,1.05<Vbus<1.1
3,"Solar = 0 MW, Hydro = 400 MW, Wind = 40 MW, HLS",211,HYDRO_G,2,20.0,1.066,21.322,1.05<Vbus<1.1
4,"Solar = 0 MW, Hydro = 400 MW, Wind = 40 MW, HLS",212,HYDRO_N,2,20.0,1.066,21.322,1.05<Vbus<1.1
...,...,...,...,...,...,...,...,...
101,"Solar = 100 MW, Hydro = 600 MW, Wind = 0 MW, HLS",212,HYDRO_N,2,20.0,1.083,21.661,1.05<Vbus<1.1
102,"Solar = 100 MW, Hydro = 600 MW, Wind = 40 MW, HLS",211,HYDRO_G,2,20.0,1.083,21.657,1.05<Vbus<1.1
103,"Solar = 100 MW, Hydro = 600 MW, Wind = 40 MW, HLS",212,HYDRO_N,2,20.0,1.083,21.657,1.05<Vbus<1.1
104,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, HLS",211,HYDRO_G,2,20.0,1.083,21.665,1.05<Vbus<1.1


##### Bus high voltage plots 

In [129]:
volt_high_fil = volt_high[(volt_high['Bus Number'] == 211) | (volt_high['Bus Number'] == 212)].reset_index(drop=True)
volt_high_f = volt_high[(volt_high['Bus Number'] != 211) & (volt_high['Bus Number'] != 212)].reset_index(drop=True)

In [130]:
alt.Chart(volt_high_f).mark_line().encode(
    alt.X('Bus Voltage(PU)', scale=alt.Scale(zero=False)),
    alt.Y('Scenario',axis = alt.Axis(labelLimit= 250, titleX=-260), sort='-x'),
    alt.Color('Bus Number:N', legend=None)).properties(height = 150, width = 280).facet('Bus Number:N',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

In [131]:
alt.Chart(volt_high_fil).mark_line().encode(
    alt.X('Bus Voltage(PU)', scale=alt.Scale(zero=False)),
    alt.Y('Scenario',axis = alt.Axis(labelLimit= 250, titleX=-260), sort='-x'),
    alt.Color('Bus Number:N', legend=None)).properties(height = 500, width = 280).facet('Bus Number:N',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

##### Voltage Lower Limits

In [132]:
volt_low = data_all_volt[data_all_volt['Bus Voltage(PU)']<0.95]
list_bus = list(volt_low['Bus Number'].unique())
list_bus

[154, 205, 3008, 103, 203, 204, 3007, 153, 3012]

In [133]:
volt_low

,Scenario,Bus Number,Bus Name,Area,Base Voltage,Bus Voltage(PU),Bus Voltage(kV),Voltage Range
90,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, RLS",154,DOWNTN,1,230.0,0.947,217.759,0.95<Vbus<0.9
118,"Solar = 0 MW, Hydro = 400 MW, Wind = 40 MW, RLS",154,DOWNTN,1,230.0,0.947,217.819,0.95<Vbus<0.9
146,"Solar = 0 MW, Hydro = 400 MW, Wind = 75 MW, RLS",154,DOWNTN,1,230.0,0.947,217.716,0.95<Vbus<0.9
174,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, HLS",154,DOWNTN,1,230.0,0.920,211.643,0.95<Vbus<0.9
179,"Solar = 0 MW, Hydro = 400 MW, Wind = 0 MW, HLS",205,SUB230,2,230.0,0.930,213.973,0.95<Vbus<0.9
...,...,...,...,...,...,...,...,...
2249,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, HLS",203,EAST230,2,230.0,0.945,217.333,0.95<Vbus<0.9
2250,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, HLS",204,SUB500,2,500.0,0.936,467.844,0.95<Vbus<0.9
2251,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, HLS",205,SUB230,2,230.0,0.917,210.963,0.95<Vbus<0.9
2261,"Solar = 100 MW, Hydro = 600 MW, Wind = 75 MW, HLS",3007,RURAL,5,230.0,0.945,217.324,0.95<Vbus<0.9


##### Bus low voltage plots 

In [134]:
volt_low_fil = volt_low[(volt_low['Bus Number'] == 153) | (volt_low['Bus Number'] == 3012)].reset_index(drop=True)
volt_low_fil_1 = volt_low[(volt_low['Bus Number'] == 103) | (volt_low['Bus Number'] == 154)|(volt_low['Bus Number'] == 205) | (volt_low['Bus Number'] == 3008)].reset_index(drop=True)

In [135]:
alt.Chart(volt_low_fil).mark_line().encode(
    alt.X('Bus Voltage(PU)', scale=alt.Scale(zero=False),axis = alt.Axis(format =".3f", grid=True)),
    alt.Y('Scenario',axis = alt.Axis(labelLimit= 250, titleX=-260), sort='x'),
    alt.Color('Bus Number:N', legend=None)).properties(height = 35, width = 280).facet('Bus Number:N',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

In [136]:
alt.Chart(volt_low_fil_1).mark_line().encode(
    alt.X('Bus Voltage(PU)', scale=alt.Scale(zero=False),axis = alt.Axis(format =".3f")),
    alt.Y('Scenario',axis = alt.Axis(labelLimit= 250, titleX=-260), sort='x'),
    alt.Color('Bus Number:N', legend=None)).properties(height = 480, width = 280).facet('Bus Number:N',columns=2).resolve_scale(x='independent', y = 'independent')

alt.FacetChart(...)

#### Conclusion 

Year 5, Topology 6 limit violation check for overloading and voltage was carried out for the three load scenarios. It was seen that
1. Overloading of greater than 80% was reported for the following transformers and branches for the listed scenarios
    * Branch 154 DOWNTN      230.00 to 203 EAST230     230.00 1
    * Branch 154 DOWNTN      230.00 to 205 SUB230      230.00 1
    * Branch 153 MID230      230.00 to 154 DOWNTN      230.00 1
    * Branch 205 SUB230      230.00 to 203 EAST230     230.00 1
    * Branch 205 SUB230      230.00 to 203 EAST230     230.00 2
    * Transformer 205 SUB230      230.00 to 206 URBGEN      18.000 1
2. The buses violating the upper voltage limit for various scenarios are
    *  Buses 206, 211, 212
3. The buses violating the lower voltage limit for various scenarios are
    *  Buses 154, 205, 3008, 103, 203, 204, 3007, 153, 3012